# 10.     Metrike performansi i tehnike validacije

(Gjorgji Noveski, JSI, Slovenija)

Ocenjivanje algoritma mašinskog učenja ključni je deo bilo kog projekta. U ovom poglavlju obradićemo standardne metrike i tehnike za evaluaciju datog modela. Počnimo sa uvozom uobičajenih ML paketa.

In [ ]:
import pandas
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, accuracy_score, recall_score, roc_curve, auc, RocCurveDisplay
np.random.seed(41)

U ovom primeru razmatraćemo jedan uobičajeni skup podataka koji se odnosi na vrste irisa. Sadrži informacije o 3 vrste irisa, Iris-setosa, Iris-virginica, Iris-versicolor. Ima 4 karakteristike koje predstavljaju dimenzije delova cveta.

In [ ]:
flower_dataset = pandas.read_csv('https://raw.githubusercontent.com/VALENCEML/eBOOK/main/EN/10/iris.csv')
flower_dataset.drop('Id', axis=1, inplace=True)

In [ ]:
flower_dataset.head()

Kako bismo obučili model mašinskog učenja, podelićemo podatke na trening i test skup. Ciljna promenljiva je kolona "Species" (vrsta), dok će ostale 4 kolone biti ulaz u model.

In [ ]:
x = flower_dataset.iloc[:, :-1]
y = flower_dataset.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.4)

## 10.1. Model

Model koji ćemo koristiti za obuku i predviđanje vrsta irisa je **Support Vector Machine** (mašina sa potpornim vektorima).

In [ ]:
model = SVC(random_state=42, probability=True)
model.fit(x_train, y_train)
y_predicted = model.predict(x_test)

### 10.1.1. Generisanje matrice konfuzije

Matrica konfuzije je jedan alat koji nam pomaže da vidimo koliko dobro model radi. Dvodimenzionalna je, jedna osa predstavlja predviđenu klasu, a druga osa predstavlja stvarnu istinitu klasu. Iz nje možemo naučiti mnoge stvari i izvesti sledeće metrike:

* True positive (tačno pozitivno)
* False positive (lažno pozitivno)
* True negative (tačno negativno)
* False negative (lažno negativno)

Ove metrike nam govore da li je uzorak podataka ispravno ili neispravno klasifikovan u pozitivnu ili negativnu klasu, respektivno. Vizuelno možemo bolje razumeti kako to funkcioniše.

In [ ]:
matrix = confusion_matrix(y_test, y_predicted)
display = ConfusionMatrixDisplay(matrix, display_labels=model.classes_)
display.plot()

## 10.2. Metrike


Postoji mnogo različitih metrika koje se mogu koristiti za merenje uspešnosti mašinskog modela učenja (klasifikatora). Na prvi pogled, mogu dati slične ili iste rezultate. Ali, u zavisnosti od problema sa kojim se suočavamo i podataka, neke metrike su pogodnije od drugih. U narednim delovima objasnićemo i pokazati razne metrike koje se mogu koristiti prilikom merenja uspešnosti modela mašinskog učenja. Važno je razumeti razlike između metrika. Kada pominjemo "pozitivan primer" mislimo na opservaciju podataka koja pripada ciljnoj klasi koju želimo da predvidimo. Na primer, ako imamo problem klasifikacije slika i trebamo klasifikovati da li slika sadrži mačku, onda se one slike koje sadrže mačku nazivaju "pozitivni primeri", a one koje ne sadrže, "negativni primeri".


## 10.3. Tačnost

Metrika tačnosti razmatra sve predikcije koje je model napravio i predstavlja deo predikcija koje je model pogodio. Što se tiče zadatka binarne klasifikacije, tačnost je deo tačnih predikcija među svim predikcijama. Važno je razumeti da, u našoj situaciji kada se bavimo skupom podataka o irisima, predviđanje da je iris tipa Iris-setosa, dok je zapravo tip irisa Iris-Virginica, ima uticaja na ovu metriku.

In [ ]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_predicted)

In [ ]:
accuracy

### 10.3.1. Tačnost osetljiva na troškove

Drugi tip metrike tačnosti je tačnost osetljiva na troškove.

Postoje situacije u kojima u našem skupu podataka imamo tako malo uzoraka ciljne klase u poređenju sa drugom klasom. Na primer, ako želimo da klasifikujemo da li pacijent ima rak ili ne, možda imamo skup podataka koji se sastoji od samo 100 uzoraka sa pacijentima koji imaju rak i 900 uzoraka bez raka. U ovim situacijama želimo reći algoritmu da je važnije biti u stanju da izdvoji pacijente sa rakom i da pogrešna klasifikacija njih treba da ima veću kaznu. To se postiže pomoću tačnosti osetljive na troškove. Dodeljujemo težine našim uzorcima podataka, veće težine uzorcima koji su nam važni i želimo da ih klasifikujemo.

In [ ]:
model = SVC(random_state=42, probability=True, class_weight='balanced')
model.fit(x_train, y_train)
y_predicted = model.predict(x_test)

Ovde treniramo model koristeći tačnost osetljivu na troškove, pružajući dodatni parametar "class_weight". Vrednost "balanced" automatski dodeljuje težine klasama obrnuto proporcionalno frekvencijama klasa.

## 10.4. Preciznost

Preciznost kao metrika uzima u obzir samo jednu klasu odjednom. Cilj joj je da utvrdi koliko su naše predikcije za određenu klasu zapravo tačne. Drugim rečima, preciznost je deo tačnih predikcija među pozitivnim <u>predikcijama</u>.
U našem scenariju, ne možemo koristiti metriku preciznosti ako želimo reći koliko dobro se model pokazao na svim klasama tipova irisa, ali možemo koristiti preciznost da kažemo koliko dobro se pokazao na svakoj klasi pojedinačno.

In [ ]:
precision = precision_score(y_true=y_test, y_pred=y_predicted, average=None)

In [ ]:
precision

## 10.5. Osetljivost (Recall)

Osetljivost, isto kao i preciznost, uzima u obzir samo jednu klasu odjednom. Osetljivost koristimo kada želimo da vidimo koliko dobro naš model detektuje ciljnu klasu. Osetljivost je frakcija tačnih predviđanja među pozitivnim <u>primerima</u>. Kada kažemo pozitivni primeri, mislimo na ciljnu klasu za naš problem.

In [ ]:
recall = recall_score(y_true=y_test, y_pred=y_predicted, average=None)
recall

## 10.6. ROC kriva

ROC (Receiver Operating Characteristic) kriva ocenjuje kvalitet klasifikatora kada klasifikator izražava verovatnoće. Na primer, umesto da klasifikator strogo izražava 1 ili 0, ako uzorak pripada ciljnoj klasi ili ne, klasifikator mora da predvidi određenu verovatnoću da uzorak pripada ciljnoj klasi. Obično je opseg verovatnoće od 0,0 (0% verovatnoće) do 1,0 (100% verovatnoće).

Kada imamo verovatnoće, definišemo prag koji navodi od koje verovatnoće nadalje su svi predviđeni uzorci klasifikovani kao ciljna klasa, a drugi nisu. Menjanjem ove vrednosti praga, broj pozitivnih i negativnih primera se menja. Ako postavimo visok prag, na primer 0,9, dobićemo mali broj pozitivnih primera, ali ćemo biti sigurniji da su to tačna predviđanja. Slično tome, ako postavimo nizak prag, bićemo sigurniji da smo tačno identifikovali veći broj pozitivnih uzoraka, ali će mnogi negativni uzorci biti pogrešno klasifikovani.

ROC kriva nam pomaže da pronađemo vrednost praga koja će najbolje raditi za nas.

Pošto ROC metrika obrađuje samo maksimalno dve klase (pozitivna i negativna klasa), moraćemo da uklonimo jednu od 3 klase koje imamo u našem iris skupu podataka. Uklonićemo primere "Iris-setosa" i sada ćemo pokušati da utvrdimo da li je primer iz vrste "Iris-versicolor".

In [ ]:
two_class_dataset = flower_dataset[flower_dataset['Species'] != "Iris-setosa"].copy()
two_class_dataset['Species'] = np.where(two_class_dataset['Species'] == 'Iris-versicolor', 1, 0)

In [ ]:
x = two_class_dataset.iloc[:, :-1]
y = two_class_dataset.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

Verovatnoće za obe klase se vraćaju, ali pošto se obe zbrajaju do 1, možemo samo uzeti ciljnu klasu koja nas zanima.

In [ ]:
model.fit(x_train, y_train)
y_predicted = model.predict_proba(x_test)[:,1]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_predicted)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='example estimator')
display.plot()

## 10.7. Unakrsna validacija (Cross validation)

U stvarnoj upotrebi, samo obučavanje i testiranje klasifikatora na skupu podataka nije dovoljno da se garantuje da će ono što vidimo iz metrika biti stvarna performansa koju ćemo dobiti. Postoji šansa da smo imali sreće kada je funkcija za podelu podataka za obuku/testiranje podelila naše podatke. Da bismo bili sigurniji u sposobnost modela da obradi nove i neviđene podatke, možemo ga više puta validirati i izabrati vreme kada je postigao najbolje rezultate. To se naziva unakrsna validacija.

K-folds unakrsna validacija razbija skup podataka na **k** manjih skupova podataka, jedan za obuku i ostale za validaciju, k puta. Nakon što se obučavanje i validacija završe, čuva rezultate validacije i ponavlja isti postupak, ali sada sa različitim podacima u skupovima podataka za obuku i validaciju. Na ovaj način, svi podaci će imati priliku da budu deo obuke i validacije.

Kada podelimo skup podataka na skupove za obuku i validaciju, moramo biti sigurni da ćemo ostaviti neki deo našeg originalnog skupa podataka za testiranje. Ovi testni podaci neće biti deo ciklusa unakrsne validacije jer, nakon što izaberemo klasifikator koji je imao najbolje rezultate iz k-iteracija, želimo da ga testiramo kako bismo dobili nepristrasnu procenu konačne performanse modela.

In [ ]:
scores = cross_val_score(model, x_train, y_train, cv=5)

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Za prethodni primer, izabrali smo broj nabora 5. Šta će se dogoditi sa rezultatom ako povećamo broj nabora?

In [ ]:
scores = cross_val_score(model, x_train, y_train, cv=11)

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))